In [1]:
'''
CNN on kaggle fake news dataset 
'''
import pandas as pd
import numpy as np
import random
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import os
def load_kagglefakenews():
  
    df = pd.read_csv('Kaggle_FakeNews/train.csv', nrows=10000, encoding='utf8')
    train_data = df['text'].values.tolist() 
    train_labels = df['label'].values.tolist() 


    combo = list(zip(train_data, train_labels))
    random.shuffle(combo)
    train_data, train_labels = zip(*combo)
    del df

    return np.asarray(train_data).tolist(), np.asarray(train_labels).tolist()

Using TensorFlow backend.
/home/lenovo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lenovo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lenovo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lenovo/.local/lib/python3.7/site-packages/tensorflow/python

In [2]:
train_data, train_labels = load_kagglefakenews()

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.utils import to_categorical
import pickle

MAX_NB_WORDS=50000 #dictionary size
MAX_SEQUENCE_LENGTH=1500 #max word length of each individual article
EMBEDDING_DIM=300 #dimensionality of the embedding vector (50, 100, 200, 300)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')

def tokenize_trainingdata(texts, labels):
    tokenizer.fit_on_texts(texts)
    pickle.dump(tokenizer, open('Models/tokenizer.p', 'wb'))

    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    labels = to_categorical(labels, num_classes=len(set(labels)))

    return data, labels, word_index



In [4]:
X, Y, word_index = tokenize_trainingdata(train_data, train_labels)    

Found 165537 unique tokens.


In [5]:
#splitting the data (90% train, 5% test, 5% validation)
train_data = X[:int(len(X)*0.9)]
train_labels = Y[:int(len(X)*0.9)]
test_data = X[int(len(X)*0.9):int(len(X)*0.95)]
test_labels = Y[int(len(X)*0.9):int(len(X)*0.95)]
valid_data = X[int(len(X)*0.95):]
valid_labels = Y[int(len(X)*0.95):]

In [6]:
def load_embeddings(word_index, embeddingsfile='wordEmbeddings/glove.6B.%id.txt' %EMBEDDING_DIM):
    embeddings_index = {}
    f = open(embeddingsfile, 'r', encoding='utf8')
    for line in f:
        
        values = line.split(' ') 
        word = values[0] 
        coefs = np.asarray(values[1:], dtype='float32') 
        embeddings_index[word] = coefs 
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:

            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    return embedding_layer
    

embedding_layer = load_embeddings(word_index)

Found 400000 word vectors.


In [7]:
from keras import Sequential, Model, Input
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Flatten, Dense, GlobalAveragePooling1D, Dropout, LSTM, CuDNNLSTM, RNN, SimpleRNN, Conv2D, GlobalMaxPooling1D
from keras import callbacks

def baseline_model(sequence_input, embedded_sequences, classes=2):
    x = Conv1D(64, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Conv1D(128, 3, activation='relu')(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(256, 2, activation='relu')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(classes, activation='softmax')(x)

    model = Model(sequence_input, preds)
    return model

In [11]:

MAX_SEQUENCE_LENGTH=1500
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

model = baseline_model(sequence_input, embedded_sequences, classes=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['acc'])

print(model.summary())

model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          epochs=25, batch_size=64)


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1500, 300)         49661400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1496, 64)          96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 299, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 297, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 58, 256)           657

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("cnn_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
model.evaluate(test_data, test_labels)

500/500 [==============================] - 2s 4ms/step


[0.21101550610600578, 0.9760000109672546]

In [11]:
yhat_probs = model.predict(test_data)

In [12]:
print(yhat_probs)

[[5.90566429e-10 1.00000000e+00]
 [9.99960780e-01 3.92453112e-05]
 [1.00000000e+00 2.81567854e-20]
 [5.61138759e-34 1.00000000e+00]
 [1.58428505e-01 8.41571510e-01]
 [9.97760296e-01 2.23971810e-03]
 [1.46493819e-15 1.00000000e+00]
 [6.88354689e-08 9.99999881e-01]
 [2.37312746e-14 1.00000000e+00]
 [1.00000000e+00 3.78446553e-25]
 [4.10817707e-37 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [9.40769543e-14 1.00000000e+00]
 [3.77630316e-10 1.00000000e+00]
 [1.00000000e+00 1.27140007e-10]
 [9.99999762e-01 2.55539220e-07]
 [9.99851465e-01 1.48563980e-04]
 [1.02186757e-04 9.99897838e-01]
 [4.51567146e-21 1.00000000e+00]
 [9.99992728e-01 7.21396054e-06]
 [8.74624410e-16 1.00000000e+00]
 [1.00000000e+00 1.43256940e-09]
 [9.84111230e-13 1.00000000e+00]
 [1.00000000e+00 2.33435382e-10]
 [7.46015658e-08 9.99999881e-01]
 [1.00000000e+00 5.68927939e-28]
 [1.87464769e-17 1.00000000e+00]
 [1.38711073e-07 9.99999881e-01]
 [1.00000000e+00 1.00469754e-11]
 [2.95941476e-07 9.99999762e-01]
 [1.166740

In [17]:
yhat_classes = []
for i in range(len(yhat_probs)):
    if(yhat_probs[i][0]<.5):
        yhat_classes.append(0.)
    else:
        yhat_classes.append(1.)
    
        

In [18]:

yhat_classes = np.array(yhat_classes)
# print(yhat_classes)


In [19]:
single_test_labels = []
for i in range(len(test_labels)):
    if(test_labels[i][0]==0):
        single_test_labels.append(0.) 
    else:
        single_test_labels.append(1.)
single_test_labels= np.array(single_test_labels)        
    
# print(single_test_labels)


In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(single_test_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(single_test_labels, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(single_test_labels, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(single_test_labels, yhat_classes)
print('F1 score: %f' % f1)


Accuracy: 0.956000
Precision: 0.960526
Recall: 0.943966
F1 score: 0.952174


In [21]:
def tokenize_text(text):
    sequences = tokenizer.texts_to_sequences(text)
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return data

In [22]:

# load json and create model
json_file = open('cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [23]:
def tokenize_text(text):
    sequences = tokenizer.texts_to_sequences(text)
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return data

In [26]:
#prediction on real time data
f1 = open('scraping/article.txt', "r")
text = f1.read()
#tokenize
tok = tokenize_text([text])
pred = model.predict(tok) # % real %fake
print(pred*100)


[[97.759926   2.2400742]]


In [16]:
#Using LSTMS

In [35]:
def LSTM_model(sequence_input, embedded_sequences, classes=2):
    x = LSTM(32,return_sequences=True)(embedded_sequences)
    x = LSTM(64,return_sequences=True)(x)
    x = LSTM(128)(x)
    x = Dense(4096,activation='relu')(x)
    x = Dense(1024,activation='relu')(x)
    preds = Dense(classes, activation='softmax')(x)

    model = Model(sequence_input, preds)
    return model

In [36]:

MAX_SEQUENCE_LENGTH=1500
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
model = LSTM_model(sequence_input, embedded_sequences, classes=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['acc'])

print(model.summary())

model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          epochs=25, batch_size=64)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1500, 300)         49661400  
_________________________________________________________________
lstm_7 (LSTM)                (None, 1500, 32)          42624     
_________________________________________________________________
lstm_8 (LSTM)                (None, 1500, 64)          24832     
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              528384    
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              4195

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnnlstm_1/CudnnRNN (defined at /home/lenovo/.local/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:517) with these attrs: [input_mode="linear_input", T=DT_FLOAT, direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true, seed=87654321, dropout=0]
Registered devices: [CPU, XLA_CPU]
Registered kernels:
  <no registered kernels>

	 [[cu_dnnlstm_1/CudnnRNN]]

Errors may have originated from an input operation.
Input Source operations connected to node cu_dnnlstm_1/CudnnRNN:
 cu_dnnlstm_1/ExpandDims_2 (defined at /home/lenovo/.local/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:488)	
 cu_dnnlstm_1/ExpandDims_1 (defined at /home/lenovo/.local/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:487)	
 cu_dnnlstm_1/transpose (defined at /home/lenovo/.local/lib/python3.7/site-packages/keras/layers/cudnn_recurrent.py:484)	
 cu_dnnlstm_1/concat_1 (defined at /ho